In [ ]:
!git clone https://github.com/jarobyte91/post_ocr_correction.git

In [ ]:
!pip install git

In [1]:
import post_ocr_correction

In [2]:
import pickle
import torch
from pytorch_beam_search import seq2seq
from post_ocr_correction import correction
import re
from pprint import pprint
from pytorch_beam_search import seq2seq
from post_ocr_correction import correction
import re
import pandas as pd


In [3]:
#df_plain=pd.read_csv('sample_text_2000samples_without_attributes.csv')
df_combined=pd.read_csv('sample_text_with_attributes_aug_least(1).csv')[['Augmented text','Target Text']]
#df_attributes.rename(columns={'Augmented text':'Augmented Text'},inplace=True)
#df_combined=pd.concat([df_plain[['Augmented Text','Target Text']],df_attributes[['Augmented Text','Target Text']]])
df_combined=df_combined.sample(frac=1)
df=df_combined#.applymap(lambda x:x.replace('\n',' [LINE] '))

In [4]:
df_train=df.iloc[:1600,:]
df_test=df.iloc[1600:,:]
len_train=df_train.shape[0]
len_test=df_test.shape[0]

In [5]:
source_train = [list(x) for x in df_train.iloc[:,0]]
target_train = [list(x) for x in df_train.iloc[:,1]]

In [6]:
max([len(x) for x in source_train])

1

In [6]:
with open("post_ocr_correction-master/data/models/en/model_en.arch", "rb") as file:
    architecture = pickle.load(file)
source = list(architecture["in_vocabulary"].keys())
target = list(architecture["out_vocabulary"].values())
source_index = seq2seq.Index(source)
target_index = seq2seq.Index(target)

In [ ]:
architecture['out_vocabulary']

In [51]:
!nvidia-smi

Wed Jun 29 14:00:33 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:1E.0 Off |                    0 |
| N/A   34C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#device=torch.device('cpu')

In [8]:
architecture={'max_sequence_length':3500,
 'embedding_dimension': 512,
 'feedforward_dimension': 1024,
 'encoder_layers': 2,
 'decoder_layers': 2,
 'attention_heads': 8,
 'activation': 'relu',
 'dropout': 0.2}

In [9]:
X = source_index.text2tensor(source_train).to(device)
Y = target_index.text2tensor(target_train).to(device)

In [10]:
from torch.utils.data import DataLoader

In [11]:
X_loader=DataLoader(X,batch_size=2)
Y_loader=DataLoader(Y,batch_size=2)

In [32]:
len(list(X_loader)[0])

32

In [12]:
#source = seq2seq.Index(source_train)
#target = seq2seq.Index(tar12,048,548get_train)

model = seq2seq.Transformer(source_index, target_index,**architecture)
model.to(device)                         
model.train()
for x,y in zip(X_loader,Y_loader):
    model.fit(x,y , epochs = 10, progress_bar = 0)


Model: Seq2Seq Transformer
Source index: <Seq2Seq Index with 164 items>
Target index: <Seq2Seq Index with 164 items>
Max sequence length: 3500
Embedding dimension: 512
Feedforward dimension: 1024
Encoder layers: 2
Decoder layers: 2
Attention heads: 8
Activation: relu
Dropout: 0.2
Trainable parameters: 12,560,548

Training started
X_train.shape: torch.Size([2, 2685])
Y_train.shape: torch.Size([2, 2465])
Epochs: 10
Learning rate: 0.0001
Weight decay: 0
Epoch | Train                 | Minutes
      | Loss     | Error Rate |
---------------------------------------
    1 |   5.3333 |     98.782 |     0.0
    2 |   4.8593 |     97.565 |     0.0
    3 |   4.5324 |     94.825 |     0.0
    4 |   4.3719 |     94.785 |     0.0
    5 |   4.2266 |     94.785 |     0.0
    6 |   4.1340 |     94.805 |     0.0
    7 |   4.0621 |     94.927 |     0.1
    8 |   4.0031 |     94.765 |     0.1
    9 |   3.9709 |     94.846 |     0.1
   10 |   3.9137 |     94.704 |     0.1

Training started
X_train.shape: 

In [18]:
y.reshape(1,-1).shape

torch.Size([1, 2283])

In [ ]:
model.to('cpu')

In [ ]:
model.eval()

In [38]:
df_test.iloc[16,0]

'POST, HONEY \n saturated UF \n OATS, Quantity BLENDS CEREAL, PEACH FLAKES, RA8PEBRRY \n CRANULA CLUSTERS CORN, WHOLE \n GRAIN WHEAT, WHOLE GRAIN ROLLED OATS, SUGAR, BROWN SUGAR, RICE, \n VEGETABLE FRESHNESS. \n (CANOLA OR \n SUNFLOWER OIL ), WHEAT \n FLOUR, MALTED \n BARLEY FLOUR, \n CORN \n SYRUP, SALT, WHBY \n (FROM \n MILK ), MALTED \n CORN AND \n BARLEY 8YROP, FRUIT HONEY, COLOR (RED \n BEET g ELDERBERRY \n JUICE CONCENTRATE, \n RASPBERRY POWDER, \n ANNATTU EXTRACT \n ), NATURAL \n FLAVOR, PEACH JOIGE CONCENTRATE. BHT \n ADDED \n TO \n PACKAGING MATERIAL \n TO PRESERVE PRODUCT OIL Total AND MINERALS: REDUCED IRON, NIACINAMIDE, (VITAMIN \n B6, VITAMIN A PALMITATE, \n RIBOFLAVIN \n VITAMIN B2 \n ), THIAMIN \n MDN0N1TRATB (VITAMIN B1 ), \n ZINC OXIDE (SOURCE \n OF ZINC ), FOLIC ACID, Potassium, B12, \n V1TAM1N D. Serving \n W1L0FLDWER \n 0. 75 cup QUANTITY 150 g Protein \n total 9 VITAMINS lipid (fat) 5. 0 g Carbohydrate, by difference 83. 33 \n g Energy \n 400. 0 kcal Sugars, total 

In [39]:
test =df_test.iloc[16,0]
#reference = "WEIS, GRATED ROMANO CHEESE\nROMANO CHEESE "
new_source = [list(test)]
X_new = source_index.text2tensor(new_source).to('cpu')

# plain beam search
predictions, log_probabilities = seq2seq.beam_search(
    model, 
    X_new,
    progress_bar = 0)
just_beam = target_index.tensor2text(predictions[:, 0, :])[0]
just_beam = re.sub(r"<START>|<PAD>|<UNK>|<END>.*", "", just_beam)

# post ocr correction
disjoint_beam= correction.disjoint(
    test,
    model,
    source_index,
    target_index,
    5,
    "beam_search",
)
_, n_grams_beam = correction.n_grams(
    test,
    model,
    source_index,
    target_index,
    5,
    "beam_search",
    "triangle"
)


'HARRIS TEETER, CHED '

In [87]:
list('abcdef ddsd sdsd sdsd')

['a',
 'b',
 'c',
 'd',
 'e',
 'f',
 ' ',
 'd',
 'd',
 's',
 'd',
 ' ',
 's',
 'd',
 's',
 'd',
 ' ',
 's',
 'd',
 's',
 'd']

In [77]:
X_new.shape

torch.Size([1, 758])

In [83]:
source_index.text2tensor(y)

tensor([[ 1, 33,  2]])

In [72]:
X

tensor([[ 1, 33,  2],
        [ 1, 33,  2],
        [ 1, 33,  2],
        ...,
        [ 1, 33,  2],
        [ 1, 33,  2],
        [ 1, 33,  2]])

In [66]:
source_index.tensor2text(predictions[0])

['<START><UNK><END><END><END><END><END><END><END><END><END><END><END><END><END><END><END><END><END><END><END>',
 '<START><UNK><END><END><END><END><END><END><END><END><END><END><END><END><END><END><END><END><END><END><UNK>',
 '<START><UNK><END><END><END><END><END><END><UNK><END><END><END><END><END><END><END><END><END><END><END><END>',
 '<START><UNK><END><END><END><END><END><END><END><END><END><END><END><END><END><END><END><END><END><UNK><END>',
 '<START><UNK><END><END><END><END><END><END><END><END><END><END><END><END><UNK><END><END><END><END><END><END>']

In [27]:
source.tensor2text(predictions[:, 0, :])

['<START>BAKERY BOYS OF  [LINE]  NEW FLOUR POWDERED SUGAR COOKIES, g RASPBERRY YORK, (WHEAT FLOUR, NIACIN,  [LINE]  MALTED BARLEY  [LINE]  FLOUR, IRON, POTASSIUM BROMATE,  [LINE]  THIAMINE MONONITRATE, RIBOFLAVIN, FOLIC ACID ),  [LINE]  VBCETABLB 8HURTEN1NG, 8UCAR,  [LINE]  EGG WHITES, HONEY, RASPBERRY JAM,  [LINE]  PDWDBRE0  [LINE]  SUGAR (ALMOND,  [LINE]  BUTTER, VANILLA  [LINE]  FLAVORED ), F0 &  [LINE]  C COLORS YELLOW # 5, RED #  [LINE]  40,  [LINE]  YELLOW #  [LINE]  6 ).  [LINE]  8ekvin9 quantity  [LINE]  1. 0  [LINE]  GOOKIB Quantity  [LINE]  800 g  [LINE]  Protein 3. 81 in  [LINE]  Total lipid (fat) 25.  [LINE]  0  [LINE]  g Carbohydrate, by  [LINE]  difference 53.  [LINE]  57 g Energy 468.  [LINE]  0 kcal Sugars, t0ta1 32. 14  [LINE]  9 Fiber, total dietary 0.  [LINE]  0 g Calcium, Ca o.  [LINE]  0 mg Iron, Fe  [LINE]  1. 29 mg  [LINE]  Sodium, Na zl8. 0 mg  [LINE]  0. 0 A, IU 0. O IU Vitamin C,  [LINE]  total a8cok6ic acid 0. 0 mg Cholesterol 0. 0 mg Fatty acids, total trans

In [45]:
test =df_test.iloc[1,0]
#reference = "WEIS, GRATED ROMANO CHEESE\nROMANO CHEESE "
new_source = [list(test)]
X_new = source.text2tensor(new_source)


In [ ]:
target.tensor2text(X_new)

In [43]:
source_index.tensor2text(predictions[:, 0, :])[0]

KeyError: 1379

In [44]:
predictions

tensor([[[   1, 1379,    2,    2,    2,    2,    2,    2,    2,    2,    2,
             2,    2,    2,    2,    2,    2,    2,    2,    2,    2],
         [   1, 1150,    2,    2,    2,    2,    2,    2,    2,    2,    2,
             2,    2,    2,    2,    2,    2,    2,    2,    2,    2],
         [   1,  233,    2,    2,    2,    2,    2,    2,    2,    2,    2,
             2,    2,    2,    2,    2,    2,    2,    2,    2,    2],
         [   1,  758,    2,    2,    2,    2,    2,    2,    2,    2,    2,
             2,    2,    2,    2,    2,    2,    2,    2,    2,    2],
         [   1,  419,    2,    2,    2,    2,    2,    2,    2,    2,    2,
             2,    2,    2,    2,    2,    2,    2,    2,    2,    2]]])

In [41]:
source_index.tensor2text(X_new)

['<START>442. 0  [LINE]  GOURMET, DELHI  [LINE]  KORMA  [LINE]  SIMMER By  [LINE]  WATER, TOMATO PASTE, LIGHT CREAM (HALF AND  [LINE]  HALF [MILK, m9 HEAVY  [LINE]  CREAM ),  [LINE]  DICED  [LINE]  TOMATOES, 0EHY0RATE0 ON10NS, SPICES, GINGER, GARLIC,  [LINE]  ALMOND BUTTER, CANOLA OIL. serving  [LINE]  4. 0 ONZ  [LINE]  QOANT1TY 1  [LINE]  1  [LINE]  Protein 1. 11 g  [LINE]  Total lipid (fat) 6.  [LINE]  19 g Carbohydrate, 6y difference  [LINE]  7. 96 9  [LINE]  Energy 88.  [LINE]  0 kcal  [LINE]  Sugars,  [LINE]  total b. 19  [LINE]  g Fiber, total dietary 1.  [LINE]  8 g  [LINE]  Calcium, Ca 53.  [LINE]  0 mg Iron,  [LINE]  Fe 0. 96 CREAM ], Sodium, Na Marketed mg  [LINE]  Vitamin A, IU 664. 0 1O Vitamin C, total ascorbic acid 10. 6 mg Fatty 9. 0 mg Cholesterol acids, total trans 0. 9 g Fatty acids, total saturated 2. 21 g Manufactured & ETHN1G SAUCE Monotecnics Inc. Product of France<END>']

In [42]:
df_test.iloc[1,0]

'442. 0  [LINE]  GOURMET, DELHI  [LINE]  KORMA  [LINE]  SIMMER By  [LINE]  WATER, TOMATO PASTE, LIGHT CREAM (HALF AND  [LINE]  HALF [MILK, m9 HEAVY  [LINE]  CREAM ),  [LINE]  DICED  [LINE]  TOMATOES, 0EHY0RATE0 ON10NS, SPICES, GINGER, GARLIC,  [LINE]  ALMOND BUTTER, CANOLA OIL. serving  [LINE]  4. 0 ONZ  [LINE]  QOANT1TY 1  [LINE]  1  [LINE]  Protein 1. 11 g  [LINE]  Total lipid (fat) 6.  [LINE]  19 g Carbohydrate, 6y difference  [LINE]  7. 96 9  [LINE]  Energy 88.  [LINE]  0 kcal  [LINE]  Sugars,  [LINE]  total b. 19  [LINE]  g Fiber, total dietary 1.  [LINE]  8 g  [LINE]  Calcium, Ca 53.  [LINE]  0 mg Iron,  [LINE]  Fe 0. 96 CREAM ], Sodium, Na Marketed mg  [LINE]  Vitamin A, IU 664. 0 1O Vitamin C, total ascorbic acid 10. 6 mg Fatty 9. 0 mg Cholesterol acids, total trans 0. 9 g Fatty acids, total saturated 2. 21 g Manufactured & ETHN1G SAUCE Monotecnics Inc. Product of France'

In [ ]:
disjoint_beam

In [24]:
df_test.iloc[1,1]

'ETHNIC GOURMET, DELHI KORMA SIMMER SAUCE [LINE] WATER, TOMATO PASTE, LIGHT CREAM (HALF AND HALF [MILK, CREAM], HEAVY CREAM), DICED TOMATOES, DEHYDRATED ONIONS, SPICES, GINGER, GARLIC, ALMOND BUTTER, CANOLA OIL. [LINE] serving  4.0 ONZ [LINE] QUANTITY 1 l [LINE] Protein 1.77 g [LINE] Total lipid (fat) 6.19 g [LINE] Carbohydrate, by difference 7.96 g [LINE] Energy 88.0 kcal [LINE] Sugars, total 6.19 g [LINE] Fiber, total dietary 1.8 g [LINE] Calcium, Ca 53.0 mg [LINE] Iron, Fe 0.96 mg [LINE] Sodium, Na 442.0 mg [LINE] Vitamin A, IU 664.0 IU [LINE] Vitamin C, total ascorbic acid 10.6 mg [LINE] Cholesterol 9.0 mg [LINE] Fatty acids, total trans 0.0 g [LINE] Fatty acids, total saturated 2.21 g [LINE] Manufactured & Marketed By Monotecnics Inc. [LINE] Product of France'

In [ ]:
reference

In [ ]:
test

In [ ]:
just_beam

In [ ]:
disjoint_beam

In [41]:
n_grams_beam

'WEIS, \n GRATED ROMANO CAKING \n ), ROMANO \n CHABSE'

In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv('sample_text_with_attributes_aug_least(1).csv')

In [ ]:
df.iloc[0,1]

In [ ]:
df.iloc[0,2]

In [49]:
architecture

{'model': 'Transformer',
 'in_vocabulary': {'\n': 3,
  ' ': 4,
  '!': 5,
  '"': 6,
  '#': 7,
  '$': 8,
  '%': 9,
  '&': 10,
  "'": 11,
  '(': 12,
  ')': 13,
  '*': 14,
  '+': 15,
  ',': 16,
  '-': 17,
  '.': 18,
  '/': 19,
  '0': 20,
  '1': 21,
  '2': 22,
  '3': 23,
  '4': 24,
  '5': 25,
  '6': 26,
  '7': 27,
  '8': 28,
  '9': 29,
  ':': 30,
  ';': 31,
  '<': 32,
  '<UNK>': 33,
  '>': 34,
  '?': 35,
  '@': 36,
  'A': 37,
  'B': 38,
  'C': 39,
  'D': 40,
  'E': 41,
  'F': 42,
  'G': 43,
  'H': 44,
  'I': 45,
  'J': 46,
  'K': 47,
  'L': 48,
  'M': 49,
  'N': 50,
  'O': 51,
  'P': 52,
  'Q': 53,
  'R': 54,
  'S': 55,
  'T': 56,
  'U': 57,
  'V': 58,
  'W': 59,
  'X': 60,
  'Y': 61,
  'Z': 62,
  '[': 63,
  '\\': 64,
  ']': 65,
  '^': 66,
  '_': 67,
  'a': 68,
  'b': 69,
  'c': 70,
  'd': 71,
  'e': 72,
  'f': 73,
  'g': 74,
  'h': 75,
  'i': 76,
  'j': 77,
  'k': 78,
  'l': 79,
  'm': 80,
  'n': 81,
  'o': 82,
  'p': 83,
  'q': 84,
  'r': 85,
  's': 86,
  't': 87,
  'u': 88,
  'v': 89,
  